# Reviewing model versions

## Introduction

This cookbook will guide you through the handling and comparison of an existing computational model in jinko.  
In particular, you will be able to retrieve two specified model versions and output an html report containing the differences  


Linked resources: [Jinko](https://jinko.ai/project/e0fbb5bb-8929-439a-bad6-9e12d19d9ae4?labels=59a37e3f-471f-4958-a519-aa9ee128f5f7).

In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
import json
from deepdiff import DeepDiff
import difflib
import re
from IPython.display import HTML
import os

folder_id = "59a37e3f-471f-4958-a519-aa9ee128f5f7"

resources_dir = os.path.normpath("resources/reviewing_model_versions")
if not os.path.exists(resources_dir):
    os.makedirs(resources_dir)

In [ ]:
# Utils functions for pretty printing


# Function to format the diff output
def diffFormatCreator(m, a, b):
    result = ""
    delete = "<span style='color:red;'>"
    end = "</span>"
    insert = "<span style='color:green;'>"

    # Loop through the opcodes and format the changes
    for tag, i1, i2, j1, j2 in m.get_opcodes():
        if tag == "replace":
            result += f' {delete}{" ".join(a[i1:i2])}{end} '
            result += f'{insert}{" ".join(b[j1:j2])}{end} '
        if tag == "delete":
            result += delete
            result += " ".join(a[i1:i2])
            result += end
        if tag == "insert":
            result += insert
            result += " ".join(b[j1:j2])
            result += end
        if tag == "equal":
            result += " ".join(a[i1:i2])
    return result


# Function to extract and format the diff keys
def formatKey(input_str, componentsHere):
    # Remove 'root' and split by the brackets
    parts = re.split(r"root|\[|\]|'|specifics|contents", input_str)

    # Get the component ID based on the index
    immutableid_here = parts[3]
    id_here = [
        component
        for component in componentsHere
        if component["immutableId"] == immutableid_here
    ][0]["id"]

    # Filter out empty strings and join the remaining parts with spaces
    formatted_str = id_here + " " + " ".join(filter(None, parts[4:]))

    return formatted_str

In [ ]:
def review_versions(
    model_name,
    model_dict,
    version_new_snapshot_id=None,
    version_old_snapshot_id=None,
    output_path=None,
):
    """
    Compare two labeled versions of a model and save the HTML report.

    Args:
        model_name (str): Name of the model to compare.
        model_dict (dict): Dictionary containing model metadata.
        version_new_snapshot_id (str, optional): Snapshot ID of the new version. Defaults to the latest labeled version.
        version_old_snapshot_id (str, optional): Snapshot ID of the old version. Defaults to the second latest labeled version.
        output_path (str, optional): Directory to save the HTML diff report. If not provided, the file will not be saved.
    """
    model_id = model_dict[model_name]["coreItemId"]
    versions = jinko.list_project_item_versions(
        model_dict[model_name]["sid"], only_labeled=True
    )

    # Determine the versions to compare
    if version_new_snapshot_id and version_old_snapshot_id:
        version_new = next(
            (
                v
                for v in versions
                if v["coreId"]["snapshotId"] == version_new_snapshot_id
            ),
            None,
        )
        version_old = next(
            (
                v
                for v in versions
                if v["coreId"]["snapshotId"] == version_old_snapshot_id
            ),
            None,
        )
    elif not version_new_snapshot_id and not version_old_snapshot_id:
        version_new, version_old = versions[:2]
    else:
        raise ValueError(
            "Both version_new_snapshot_id and version_old_snapshot_id must be provided."
        )

    if not version_new or not version_old:
        raise ValueError("Specified versions were not found.")

    # Extract snapshot IDs and labels
    snapshot_id_new = version_new["coreId"]["snapshotId"]
    snapshot_id_old = version_old["coreId"]["snapshotId"]

    label_new = version_new.get("label", f"snapshot_{snapshot_id_new}")
    label_old = version_old.get("label", f"snapshot_{snapshot_id_old}")

    # Download the model versions
    model_new = jinko.download_model_interface(
        model_core_item_id=model_id, model_snapshot_id=snapshot_id_new
    )
    model_old = jinko.download_model_interface(
        model_core_item_id=model_id, model_snapshot_id=snapshot_id_old
    )

    # Generate HTML diff header
    html_diff = (
        f"<h1>Comparing {model_name}: {label_new} vs {label_old}</h1>"
        f"<p>Tags:<br>{'<br>'.join(f'{tag['id']}: {tag['immutableId']}' for tag in model_new['tags'])}</p>"
    )

    # Compare components
    for key, new_components in model_new["components"].items():
        old_components = model_old["components"].get(key, [])

        # Remove diagnostics from comparison
        for comp in old_components + new_components:
            comp.pop("diagnostics", None)

        diff = DeepDiff(
            old_components, new_components, ignore_order=True, group_by="immutableId"
        )
        if diff:
            html_diff += f"<h2>Changes in {key}</h2>"

            # Values changed
            for change, details in diff.get("values_changed", {}).items():
                old_val = details["old_value"]
                new_val = details["new_value"]
                diff_html = diffFormatCreator(
                    difflib.SequenceMatcher(
                        a=str(old_val).split(), b=str(new_val).split()
                    ),
                    str(old_val).split(),
                    str(new_val).split(),
                )
                html_diff += f"<p>Changed {formatKey(change, old_components)}:<br>{diff_html}</p>"

            # Items added
            for added in diff.get("dictionary_item_added", []):
                item_id = re.sub(r"root|\[|\]|'", "", added)
                new_item = next(
                    c for c in new_components if c["immutableId"] == item_id
                )
                html_diff += f"<p>Added {formatKey(added, new_components)}:<br>{json.dumps(new_item, indent=4).replace('\n', '<br>').replace(' ', '&nbsp;')}</p>"

            # Items removed
            for removed in diff.get("dictionary_item_removed", []):
                item_id = re.sub(r"root|\[|\]|'", "", removed)
                old_item = next(
                    c for c in old_components if c["immutableId"] == item_id
                )
                html_diff += f"<p>Removed {formatKey(removed, old_components)}:<br>{json.dumps(old_item, indent=4).replace('\n', '<br>').replace(' ', '&nbsp;')}</p>"

    # Save and display the HTML diff
    if output_path:
        file_path = f"{output_path}/{model_name}_{label_old}_vs_{label_new}_diff.html"
        with open(file_path, "w") as file:
            file.write(html_diff)

    display(HTML(html_diff))

## Step 1 : Display all available models

In [ ]:
model_dict = jinko.get_models_in_folder(folder_id)

print("All models in the folder:")
print(json.dumps(model_dict, indent=4))

## Step 2 : Display all labeled versions of a model

In [ ]:
MODEL_NAME = "Test-Model"

model_versions = jinko.list_project_item_versions(
    model_dict[MODEL_NAME]["sid"], only_labeled=True
)

print(
    "All "
    + str(len(model_versions))
    + " labeled model versions available for "
    + str(MODEL_NAME)
    + " are:"
)
print(json.dumps(model_versions, indent=4))

## Step 3 : Compute the differences between 2 model versions

In the following cell, after taking care of formatting in the two helper functions *diffFormatCreator* and *formatKey*, we set up the function *reviewVersions* that will:
- download specified versions of the model (in the readable format outputted by the model editor, closely corresponding to the view on jinko)
- generate an HTML report of the differences of these two model versions using the packages difflib and deepdiff, with a section for each type of model component (parameters, species, reactions, ...)

The output folder for the downloaded models and the generated HTML report is an input to this function.

In [ ]:
# Default review using the last two labeled versions
review_versions("Test-Model", model_dict, output_path=resources_dir)

In [ ]:
seventh_version_snapshot_id = next(
    (
        version["coreId"]["snapshotId"]
        for version in model_versions
        if version["label"] == "SeventhVersion"
    ),
    None,
)
second_version_snapshot_id = next(
    (
        version["coreId"]["snapshotId"]
        for version in model_versions
        if version["label"] == "SecondVersion"
    ),
    None,
)

# Reviews for specified versions
review_versions(
    "Test-Model",
    model_dict,
    version_new_snapshot_id=seventh_version_snapshot_id,
    version_old_snapshot_id=second_version_snapshot_id,
    output_path=resources_dir,
)